In [3]:
import geopandas as gpd
import pandas as pd
# 读取grids.geojson文件
grid = gpd.read_file('grids.geojson')
# 读取data
data = pd.read_csv('population_density_split.csv')

In [47]:
# 检查是否存在几何列
print(grid.columns)

# 如果没有正确加载几何列，或者列名不是 'geometry'，你需要显式设置几何列
# 比如，如果你的几何列的名字是 'geom'，就需要用 'geom' 替代 'geometry'
grid = grid.set_geometry('geometry')  # 或者使用 grid.set_geometry('geom')，具体根据你的数据列名
print(grid.columns)

# 确保设置了几何列后，再创建空间索引
grid.sindex  # 创建空间索引

# 打印一下几何列和空间索引，确保它们都设置正确
print(grid.geometry.head())
print(grid.sindex)

Index(['geometry'], dtype='object')
Index(['geometry'], dtype='object')
0    POLYGON ((97.3666 26.04763, 97.3666 26.05662, ...
1    POLYGON ((97.3666 26.05662, 97.3666 26.0656, 9...
2    POLYGON ((97.3666 26.0656, 97.3666 26.07458, 9...
3    POLYGON ((97.3666 26.07458, 97.3666 26.08356, ...
4    POLYGON ((97.3666 26.08356, 97.3666 26.09255, ...
Name: geometry, dtype: geometry


In [34]:

from shapely.geometry import Point


# 第二步：创建 'geometry' 列，这里将 'X' 和 'Y' 列转换为 Point 对象
geometry = [Point(xy) for xy in zip(data['X'], data['Y'])]

# 第三步：将数据转换为 GeoDataFrame
gdf = gpd.GeoDataFrame(data, geometry=geometry)

# 第四步：设置坐标参考系统（CRS），假设数据是 WGS84 (EPSG:4326)
gdf.crs = 'EPSG:4326'

# 打印 GeoDataFrame 的前几行，确保数据被正确转换
print(gdf.head())

# 如果 CRS 不一致，可以使用 `to_crs()` 来转换 CRS
# gdf = gdf.to_crs('EPSG:4326')


           X          Y    Z                   geometry
0  96.652916  34.637083  0.0  POINT (96.65292 34.63708)
1  96.661250  34.637083  0.0  POINT (96.66125 34.63708)
2  96.669583  34.637083  0.0  POINT (96.66958 34.63708)
3  96.677916  34.637083  0.0  POINT (96.67792 34.63708)
4  96.686250  34.637083  0.0  POINT (96.68625 34.63708)


In [4]:
import sqlite3
import numpy as np



poi_density = []
poi_diversity = []

def get_db():
    conn = sqlite3.connect('geovis.sqlite')
    return conn

conn = get_db()
cur = conn.cursor()

In [20]:
def cal_max_min_population(population_density):
    print("计算population最大最小值")
    # 计算四个指标的最大最小值
    # 人口密度

    # 将数据转换为NumPy数组以加快处理速度
    data_array = data[['X', 'Y', 'Z']].values

    for i in range(grid.shape[0]):
        start_lon = list(grid.loc[i,'geometry'].exterior.coords)[2][0]
        start_lat = list(grid.loc[i,'geometry'].exterior.coords)[0][1]
        end_lon = list(grid.loc[i,'geometry'].exterior.coords)[0][0]
        end_lat = list(grid.loc[i,'geometry'].exterior.coords)[2][1]

        # 使用NumPy进行矢量化筛选
        mask = (data_array[:, 0] >= start_lon) & (data_array[:, 0] <= end_lon) & (data_array[:, 1] >= start_lat) & (data_array[:, 1] <= end_lat)
        filtered_data = data_array[mask]

        if filtered_data.size > 0:
            mean_value = np.mean(filtered_data[:, 2])
            population_density.append(mean_value)
        else:
            population_density.append(np.nan)

        if(i % 10000 == 0):
            print("已完成{}个网格".format(i))

    # 去除NaN值后计算最大最小值
    # population_density = [x for x in population_density if not np.isnan(x)]
    # print("人口密度最大值：",max(population_density))
    # print("人口密度最小值：",min(population_density))

In [21]:
population_density = []
cal_max_min_population(population_density)
# 去除NaN值后计算最大最小值
population_density = [x for x in population_density if not np.isnan(x)]
print("人口密度最大值：",max(population_density))
print("人口密度最小值：",min(population_density))

计算population最大最小值
已完成0个网格
已完成10000个网格
已完成20000个网格
已完成30000个网格
已完成40000个网格
已完成50000个网格
已完成60000个网格
已完成70000个网格
已完成80000个网格
已完成90000个网格
已完成100000个网格
已完成110000个网格
已完成120000个网格
已完成130000个网格
已完成140000个网格
已完成150000个网格
已完成160000个网格
已完成170000个网格
已完成180000个网格
已完成190000个网格
已完成200000个网格
已完成210000个网格
已完成220000个网格
已完成230000个网格
已完成240000个网格
已完成250000个网格
已完成260000个网格
已完成270000个网格
已完成280000个网格
已完成290000个网格


KeyboardInterrupt: 

In [ ]:
print(population_density)

In [117]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

def cal_max_min_population_sindex(grid, data,population_density):
    print("计算population最大最小值")
    print(grid.columns)
    # 为网格数据建立空间索引
    grid = grid.set_geometry('geometry')
    print(grid.columns)
    grid.sindex  # 创建空间索引
    print(grid.sindex)
    # 将数据转换为GeoDataFrame
    data_gdf = gpd.GeoDataFrame(data, geometry=[Point(xy) for xy in zip(data['X'], data['Y'])])
    data_gdf.crs = grid.crs  # 确保坐标参考系一致

    # 明确指定几何列
    data_gdf = data_gdf.set_geometry('geometry')

    # 遍历网格，使用空间索引进行筛选
    for i, row in grid.iterrows():
        # 获取当前网格的几何边界
        grid_polygon = row['geometry']

        # # 打印网格的边界信息，调试用
        # print(f"处理第 {i} 个网格，边界：", grid_polygon.bounds)

        # 使用空间索引查找可能在当前网格内的点
        possible_matches_index = list(data_gdf.sindex.intersection(grid_polygon.bounds))
        # print(f"可能的匹配项索引：{possible_matches_index}")
        
        # 提取这些候选数据
        possible_matches = data_gdf.iloc[possible_matches_index]
        # print("possible matches:",possible_matches)

        # 筛选在当前网格内的点
        filtered_data = possible_matches[possible_matches.geometry.within(grid_polygon)]

        if not filtered_data.empty:
            # 计算人口密度的平均值
            mean_value = np.mean(filtered_data['Z'])
            population_density.append(mean_value)
        else:
            population_density.append(0)
        
        if i % 100000 == 0:
            print(f"已完成 {i} 个网格")
    
    return population_density


In [119]:
population_density = []
population_density = cal_max_min_population_sindex(grid,data,population_density)
# 去除NaN值后计算最大最小值
# population_density = [x for x in population_density if not np.isnan(x)]
# print(population_density)
print("人口密度最大值：",max(population_density)) #165134.453125
print("人口密度最小值：",min(population_density))#0

计算population最大最小值
Index(['geometry'], dtype='object')
Index(['geometry'], dtype='object')
已完成 0 个网格
已完成 100000 个网格
已完成 200000 个网格
已完成 300000 个网格
已完成 400000 个网格
已完成 500000 个网格
已完成 600000 个网格
已完成 700000 个网格
已完成 800000 个网格
已完成 900000 个网格
人口密度最大值： 165134.453125
人口密度最小值： 0


In [121]:
print(population_density.count(0)) # 0的个数

102858


In [123]:
np.save('population_density.npy',population_density)

In [110]:
def cal_max_min_poi(grid, poi_density, poi_diversity, canyin_df, company_df, mall_df, jpbank_df, yzbank_df,express_df):
    print("开始计算POI密度和多样性")
    
    # 确保网格索引是连续的整数
    grid = grid.reset_index(drop=True)
    
    for i in range(grid.shape[0]):
        # 获取当前网格的几何边界
        grid_polygon = grid.loc[i, 'geometry']
        min_lon, min_lat, max_lon, max_lat = grid_polygon.bounds

        # 筛选每个POI数据，获取在当前网格内的POI
        canyin_count = canyin_df[(canyin_df['lon'] >= min_lon) & (canyin_df['lon'] <= max_lon) &
                                  (canyin_df['lat'] >= min_lat) & (canyin_df['lat'] <= max_lat)].shape[0]
        company_count = company_df[(company_df['lon'] >= min_lon) & (company_df['lon'] <= max_lon) &
                                    (company_df['lat'] >= min_lat) & (company_df['lat'] <= max_lat)].shape[0]
        mall_count = mall_df[(mall_df['lon'] >= min_lon) & (mall_df['lon'] <= max_lon) &
                              (mall_df['lat'] >= min_lat) & (mall_df['lat'] <= max_lat)].shape[0]
        jpbank_count = jpbank_df[(jpbank_df['lon'] >= min_lon) & (jpbank_df['lon'] <= max_lon) &
                                  (jpbank_df['lat'] >= min_lat) & (jpbank_df['lat'] <= max_lat)].shape[0]
        yzbank_count = yzbank_df[(yzbank_df['lon'] >= min_lon) & (yzbank_df['lon'] <= max_lon) &
                                  (yzbank_df['lat'] >= min_lat) & (yzbank_df['lat'] <= max_lat)].shape[0]
        bank_count = jpbank_count + yzbank_count
        express_count = express_df[(express_df['lon'] >= min_lon) & (express_df['lon'] <= max_lon) &
                                    (express_df['lat'] >= min_lat) & (express_df['lat'] <= max_lat)].shape[0]
        
        # 计算总POI数量
        poi_count = canyin_count + company_count + mall_count + bank_count + express_count
        
        # 计算POI密度（单位：POI数量/平方千米）
        # 假设网格为1平方千米
        poi_density_value = poi_count / 1  # 1平方千米
        
        # 计算POI多样性
        if poi_count == 0:
            poi_diversity_value = 0
        else:
            poi_diversity_value = -sum([x / poi_count * np.log(x / poi_count) 
                                        for x in [canyin_count, company_count, mall_count, bank_count, express_count] if x != 0])

        # 保存计算结果
        poi_density.append(poi_density_value)
        poi_diversity.append(poi_diversity_value)

        if i % 100000 == 0:
            print(f"已完成 {i} 个网格")

    return poi_density, poi_diversity




In [106]:
# 示例数据加载（假设每个CSV文件有lon, lat列）
canyin_df = pd.read_csv('canyin.csv')
company_df = pd.read_csv('Company.csv')
mall_df = pd.read_csv('Mall.csv')
jpbank_df = pd.read_csv('jpBank.csv')
yzbank_df = pd.read_csv('newYZBank.csv')
express_df = pd.read_csv('express.csv')

In [111]:
# 使用网格数据计算POI密度和多样性
poi_density = []
poi_diversity = []

poi_density, poi_diversity = cal_max_min_poi(grid, poi_density, poi_diversity, canyin_df, company_df, mall_df, jpbank_df, yzbank_df,express_df)

# 输出结果
print("POI密度最大值：", max(poi_density))#1333
print("POI密度最小值：", min(poi_density))#0
print("POI多样性最大值：", max(poi_diversity))#1.5498260458782016
print("POI多样性最小值：", min(poi_diversity))#0

开始计算POI密度和多样性
已完成 0 个网格
已完成 100000 个网格
已完成 200000 个网格
已完成 300000 个网格
已完成 400000 个网格
已完成 500000 个网格
已完成 600000 个网格
已完成 700000 个网格
已完成 800000 个网格
已完成 900000 个网格
POI密度最大值： 1333.0
POI密度最小值： 0.0
POI多样性最大值： 1.5498260458782016
POI多样性最小值： 0


In [112]:
print(len(poi_density))
print(len(poi_diversity))

989000
989000


In [113]:
np.save('poi_density.npy', poi_density)
np.save('poi_diversity.npy', poi_diversity)

In [124]:
# min-max标准化
def min_max_scaler(data):
    data = (data - np.min(data)) / (np.max(data) - np.min(data))
    return data

In [125]:
scale_population_density = min_max_scaler(population_density)
np.save('scale_population_density.npy', scale_population_density)

In [126]:
population_density_prob = []
sum_population_density = np.sum(scale_population_density)
# 计算概率矩阵
for i in range(len(scale_population_density)):
    population_density_prob.append(scale_population_density[i] / sum_population_density)

np.save('prob_population_density.npy', population_density_prob)

In [128]:
# 计算信息熵
def cal_entropy(prob):
    entropy = 0
    for i in range(len(prob)):
        if prob[i] != 0:
            entropy -= prob[i] * np.log(prob[i])
    # 变为0-1之间
    entropy /= np.log(len(prob))
    return entropy

In [129]:
population_density_entropy = cal_entropy(population_density_prob)
print("人口密度信息熵：", population_density_entropy) #0.8673254174870703

人口密度信息熵： 0.8673254174870703


In [115]:

scale_poi_density = min_max_scaler(poi_density)
scale_poi_diversity = min_max_scaler(poi_diversity)

np.save('scale_poi_density.npy', scale_poi_density)
np.save('scale_poi_diversity.npy', scale_poi_diversity)

In [ ]:
poi_density_prob = []
poi_diversity_prob = []
sum_poi_density = np.sum(scale_poi_density)
sum_poi_diversity = np.sum(scale_poi_diversity)
# 计算概率矩阵
for i in range(len(scale_poi_density)):
    poi_density_prob.append(scale_poi_density[i] / sum_poi_density)

for i in range(len(scale_poi_diversity)):
    poi_diversity_prob.append(scale_poi_diversity[i] / sum_poi_diversity)

np.save('prob_poi_density.npy', poi_density_prob)
np.save('prob_poi_diversity.npy', poi_diversity_prob)